# Create convolutional neural network model

## Imports

In [ ]:
from tflearn import DNN
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_augmentation import ImageAugmentation
from tflearn.data_utils import image_preloader
import numpy as np

## Get dataset

In [ ]:
# Load path/class_id image file:
data_folder = 'images/crosslight'

# Build the preloader array, resize images to 320 x 240
X, y = image_preloader(data_folder, image_shape=(128, 128), mode='folder', categorical_labels=True, normalize=True, files_extension=['.jpg'])

# Show shape and features
print("image shape: ", X[0].shape, "Features: ", len(y[0]))

from matplotlib.pyplot import imshow
%matplotlib inline

imshow(X[79])

# Create train and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

### Image augmentation
We use image augmentation to create a better training effect for real world situations. We randomly blur images and rotate them.

In [ ]:
# Real-time data augmentation (This is only used while training the DNN)
img_aug = ImageAugmentation()
img_aug.add_random_blur(0.10)
img_aug.add_random_rotation(max_angle=15.0)

### Convolutional neural network model

In [ ]:
# Convolutional network building
network = input_data(shape=[None, 64, 64, 3],
                     data_augmentation=img_aug)
network = conv_2d(network, 16, 3, activation='relu', name="conv2d-1")
network = max_pool_2d(network, 2)
network = conv_2d(network, 16, 3, activation='relu', name="conv2d-2")
network = max_pool_2d(network, 2)
network = fully_connected(network, 32, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 32, activation='relu')
network = fully_connected(network, 2, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

model = DNN(network, tensorboard_verbose=0)

### Train the model and save it

In [ ]:
#model.load('models/model_crosslight')

In [ ]:
# Start the trainig process
model.fit(X_train, y_train, n_epoch=100, shuffle=True, validation_set=0.1, show_metric=True, batch_size=8, validation_batch_size=32, run_id='ampel')

In [ ]:
from tflearn.helpers.evaluator import Evaluator

In [ ]:
# Evaluate performance on test set
model.evaluate(X_test, y_test,  batch_size=8)

In [ ]:
#print(np.around(model.predict(X_test), 3))

In [ ]:
#print(y_test)

In [ ]:
# Save the model
model.save('models/model_crosslight')

# Close the database connection
#h5f.close()